# Off-Design Missions

## Overview

Off-design missions are enabled for the following mission types:

* height_energy (collocation)    (completed)
* 2DOF (collocation) (complete)
* 2DOF (shooting)    (future work)
* height_energy (shooting)    (future work)

An off-design mission can be created either by directly entering the parameters of the sized aircraft or by using the built-in functionality to run an off-design in the same script as the sizing mission.
There are currently two types of off-design missions supported in Aviary; alternate and fallout.
For an alternate mission, the target range of the mission and the payload mass are specified and the fuel mass required is solved for.
Fallout missions meanwhile take the payload and fuel masses as inputs and determine the range of the aircraft.
To create a simple off-design mission, take your input deck and add the line `settings:problem_type, fallout` or `settings:problem_type, alternate`.

```{note}
You may need to revise some of the values in your input deck for the off-design mission.
Since the aircraft is not re-designed, it is assumed that provided inputs constitute a valid aircraft.
```

From there, the mission may be run as before.

You may also run multiple off-design missions in the same script used to solve the design mission.
An examples of this is shown in `run_off_design_example.py`.

In [ ]:
# Testing Cell
from aviary.docs.tests.utils import check_contains
import aviary.api as av
import os

off_design_examples = av.get_path(os.path.join('examples'))
check_contains(
    ('run_off_design_example.py'),
    os.listdir(off_design_examples),
    error_string="{var} not in "+str(off_design_examples),
    error_type=FileNotFoundError)